## matplotlib (plotting) and SciPy (scientific algorithms) libraries 

The SciPy library provides an emormous variety of scientific algorithms, exposed in the Python language,
but often written in low-level languages like C and Fortran to empower us with the fastest performance. 
Many of these algorithms are too "heavy" to be implemented in the basic NumPy library; however, SciPy
is built upon the data structures and operations enabled by NumPy and the two libraries are often
used side-by-side.

Let's start with a common problem in many scientific disciplines -- calculating the Voronoi diagram
for a set of data points. Since this problem falls into the category of "computational geometry,"
the SciPy developers have placed it in the scipy.spatial submodule with other spatial algorithms
and data structures.

In [ ]:
import numpy as np
import scipy
import scipy.spatial
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# we start off with a set of random points, which may for example represent
# standing trees in a forest after a forest fire event
tree_positions = np.random.randn(15, 2)

In [ ]:
# at this point, it is already helpful to first take a look at the positions of the trees on the 2D "map"
# let's start using matplotlib for this work

# first generate a figure object
fig = plt.figure()

# then create an axis object (which can be used for adding the data and labels)
ax = fig.add_subplot(111) # 111 means 1 row, 1 column, 1st plot in our figure "panel"

# scatter plot the 2D tree position data
ax.scatter(tree_positions[...,0], # x coordinates
           tree_positions[...,1]) # y coordinates

ax.set_title("Trees Remaining After Forest Fire")
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
# The Voronoi diagram will tell us which parts of the forest are closest to which tree
# This can be used i.e., as an estimate of the amount of area occupied by each tree (the area around it)

vor = scipy.spatial.Voronoi(tree_positions)

# it is such a common operation to plot the Voronoi diagram of a set of 2D generators
# that SciPy can plot the diagram (using matplotlib under the hood) directly
# from the vor object above

fig_vor = scipy.spatial.voronoi_plot_2d(vor)